In [41]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

將17個data合併，把data中'Pressure(hpa)'>1050的資料改成1050，'Humidity(%)'>100的資料改成100，並存成新的檔案datanew，捨去資料中'WindSpeed(m/s)'欄位

In [42]:
#讀取檔案
datanew = pd.read_csv('./data/L_data/datanew.csv')
datanew.shape

(1375028, 8)

In [43]:
#不要datanew中'WindSpeed(m/s)'欄位
datanew.drop(['WindSpeed(m/s)'],axis=1,inplace=True)
datanew.columns

Index(['LocationCode', 'DateTime', 'Pressure(hpa)', 'Temperature(°C)',
       'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)'],
      dtype='object')

定義'Power'反歸一化函數，並設定歸一化條件

In [44]:
# 預先計算最大和最小值
max_power = datanew['Power(mW)'].max()
min_power = datanew['Power(mW)'].min()
range_power = max_power - min_power
# 預先計算最大和最小值
print(range_power)
print(min_power)
print(max_power)

# 定義函數，使用預計算的最大值和最小值
def inverse_transform_power(value):
    return value * range_power + min_power

# 定義函數，使用預計算的最大值和最小值
def inverse_transform_power(value):
    return value * 2626.48 + 0.0

2626.48
0.0
2626.48


In [45]:
#對datanew中的'Pressure(hpa)','Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)'做歸一化
name="_maxmin"
if name=="_maxmin":
    scaler = MinMaxScaler()
    datanew[['Pressure(hpa)','Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)']] = scaler.fit_transform(datanew[['Pressure(hpa)','Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)']])
    datanew

去掉07:00以前和17:00以後的資料，並增加'Date'、'Time'、'TimeMin'欄位，以'TimeMin'時分計算'TimeMinGroup'欄位，區分該時分所屬時間區段

In [46]:
#將data_1按照DateTime排序
data_1 = datanew.sort_values('DateTime')
data_1.head(3)

,LocationCode,DateTime,Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW)
1278386,17,2024-01-01 06:29:01.000,0.705063,0.265886,0.583380,0.000021,0.000000
1278387,17,2024-01-01 06:30:01.000,0.706151,0.265886,0.583380,0.000078,0.000000
1278388,17,2024-01-01 06:31:01.000,0.706696,0.265886,0.584509,0.000127,0.000004


In [47]:
# 將 DateTime 欄位轉換為 datetime 格式
data_1['DateTime'] = pd.to_datetime(data_1['DateTime'])

# 過濾條件：僅保留 07:00:00 - 17:00:00 的資料
filtered_data = data_1[(data_1['DateTime'].dt.time >= pd.to_datetime('07:00:00').time()) &
                       (data_1['DateTime'].dt.time <= pd.to_datetime('16:59:59').time())]

#對filtered_data創建新欄位"Date"，並將DateTime欄位中的日期部分取出存入Date欄位
filtered_data['Date'] = filtered_data['DateTime'].dt.date

#對filtered_data創建新欄位"Time"，並將DateTime欄位中的時間部分取出存入Time欄位
filtered_data['Time'] = filtered_data['DateTime'].dt.time

#對filtered_data創建新欄位"TimeMin"，並將DateTime欄位中的時間部分(時分)取出存入TimeMin欄位
filtered_data['TimeMin'] = filtered_data['DateTime'].dt.strftime('%H:%M')

/tmp/ipykernel_3027225/2084215928.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Date'] = filtered_data['DateTime'].dt.date
/tmp/ipykernel_3027225/2084215928.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Time'] = filtered_data['DateTime'].dt.time
/tmp/ipykernel_3027225/2084215928.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [48]:
# 定義一個函數以便根據10分鐘區間來確定 "TimeMinGroup" 值
def get_time_min_group(time_str):
    hour, minute = map(int, time_str.split(':'))
    minute_group = (minute // 10) * 10  # 獲得10分鐘區間的開始時間
    return f"{hour:02d}:{minute_group:02d}"

# 套用函數來創建 "TimeMinGroup"
filtered_data['TimeMinGroup'] = filtered_data['TimeMin'].apply(get_time_min_group)

filtered_data.head(3)

/tmp/ipykernel_3027225/4117461723.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['TimeMinGroup'] = filtered_data['TimeMin'].apply(get_time_min_group)


,LocationCode,DateTime,Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW),Date,Time,TimeMin,TimeMinGroup
1278417,17,2024-01-01 07:00:01,0.708111,0.267559,0.598058,0.005075,0.000049,2024-01-01,07:00:01,07:00,07:00
29,1,2024-01-01 07:00:07,0.637343,0.259197,0.852094,0.003362,0.000027,2024-01-01,07:00:07,07:00,07:00
1278418,17,2024-01-01 07:01:01,0.708002,0.267559,0.601445,0.006271,0.000053,2024-01-01,07:01:01,07:01,07:00


計算各時間區段變項平均值，並將缺少的各'LocationCode', 'Date'之'TimeMinGroup'補足

In [49]:
# 按 LocationCode, Date, 和 TimeMinGroup 分組，並計算各變項的平均值
grouped_result = filtered_data.groupby(['LocationCode', 'Date', 'TimeMinGroup']).agg(
    {
        'Pressure(hpa)': 'mean',
        'Temperature(°C)': 'mean',
        'Humidity(%)': 'mean',
        'Sunlight(Lux)': 'mean',
        'Power(mW)': 'mean'
    }
).reset_index()

grouped_result.head(3)

,LocationCode,Date,TimeMinGroup,Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW)
0,1,2024-01-01,07:00,0.637779,0.259197,0.853111,0.003829,0.000032
1,1,2024-01-01,07:10,0.638258,0.260368,0.852885,0.005433,0.000053
2,1,2024-01-01,07:20,0.639869,0.261371,0.843965,0.007386,0.000094


In [50]:
#看grouped_result中TimeMinGroup有哪些值
grouped_result['TimeMinGroup'].unique()

array(['07:00', '07:10', '07:20', '07:30', '07:40', '07:50', '08:00',
       '08:10', '08:20', '08:30', '08:40', '08:50', '09:00', '09:10',
       '09:20', '09:30', '09:40', '09:50', '10:00', '10:10', '10:20',
       '10:30', '10:40', '10:50', '11:00', '11:10', '11:20', '11:30',
       '11:40', '11:50', '12:00', '12:10', '12:20', '12:30', '12:40',
       '12:50', '13:00', '13:10', '13:20', '13:30', '13:40', '13:50',
       '14:00', '14:10', '14:20', '14:30', '14:40', '14:50', '15:00',
       '15:10', '15:20', '15:30', '15:40', '15:50', '16:00', '16:10',
       '16:20', '16:30', '16:40', '16:50'], dtype=object)

In [51]:
# 定義時間區間，從 07:00 到 16:50，每隔10分鐘
time_range = pd.date_range("07:00", "16:50", freq="10T").strftime('%H:%M')

# 獲取唯一的 LocationCode 和 Date 組合
unique_location_dates = grouped_result[['LocationCode', 'Date']].drop_duplicates()

# 擴展 DataFrame，以包含每個 LocationCode 和 Date 的完整 TimeMinGroup
expanded_result = pd.DataFrame(
    [
        {'LocationCode': loc, 'Date': date, 'TimeMinGroup': time}
        for loc, date in unique_location_dates.itertuples(index=False)
        for time in time_range
    ]
)

# 與 grouped_result 合併，補齊缺少的時間段，並對缺失數據填入 NaN
final_result = pd.merge(expanded_result, grouped_result, on=['LocationCode', 'Date', 'TimeMinGroup'], how='left')

final_result.head(3)

#將final_result存成csv檔
final_result.to_csv(f'./data/preprocess_data/TimeMinGroupData{name}.csv',index=False)

/tmp/ipykernel_3027225/3966785995.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  time_range = pd.date_range("07:00", "16:50", freq="10T").strftime('%H:%M')


計算各LocationCode中Date的最小和最大值，將中間缺少的Date和各Date的'TimeMinGroup'補足

In [52]:
final_result = pd.read_csv(f'./data/preprocess_data/TimeMinGroupData{name}.csv')
final_result.head(3)

,LocationCode,Date,TimeMinGroup,Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW)
0,1,2024-01-01,07:00,0.637779,0.259197,0.853111,0.003829,0.000032
1,1,2024-01-01,07:10,0.638258,0.260368,0.852885,0.005433,0.000053
2,1,2024-01-01,07:20,0.639869,0.261371,0.843965,0.007386,0.000094


In [53]:
# 將 Date 列轉換為 datetime 類型
final_result['Date'] = pd.to_datetime(final_result['Date'])

# 取得唯一的 TimeMinGroup 值列表
time_min_groups = final_result['TimeMinGroup'].unique()

# 初始化一個空的 DataFrame 用來存放擴展後的數據
expanded_df = pd.DataFrame()

# 對每個 LocationCode 生成從最小到最大的日期範圍
for location in final_result['LocationCode'].unique():
    location_data = final_result[final_result['LocationCode'] == location]
    min_date = location_data['Date'].min()
    max_date = location_data['Date'].max()
    
    # 生成完整的日期範圍
    full_dates = pd.date_range(start=min_date, end=max_date, freq='D')
    
    # 創建每個日期和 TimeMinGroup 的多重索引
    location_df = pd.DataFrame(
        index=pd.MultiIndex.from_product([full_dates, time_min_groups], names=['Date', 'TimeMinGroup'])
    ).reset_index()
    
    # 指定 LocationCode
    location_df['LocationCode'] = location
    
    # 將原始數據合併進去，保留已有數據，缺失的部分以 NaN 補充
    location_df = location_df.merge(
        location_data,
        on=['LocationCode', 'Date', 'TimeMinGroup'],
        how='left'
    )
    
    # 將結果添加到擴展 DataFrame 中
    expanded_df = pd.concat([expanded_df, location_df], ignore_index=True)

# 重新排序欄位順序以符合原始結構
expanded_df = expanded_df[['LocationCode', 'Date', 'TimeMinGroup', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)']]

#將expanded_df存成csv檔
expanded_df.to_csv(f'./data/preprocess_data/TimeMinGroupData_FullDate{name}.csv',index=False)

針對每一個'LocationCode'，從'Date'中的最小值開始到最大值，將每一天的每一個'TimeMinGroup'的'Power(mW)'重新整理成每個row都是一筆連續兩天每個時段的值

In [54]:
# 取得 TimeMinGroup 排序
time_min_groups_sorted = sorted(expanded_df['TimeMinGroup'].unique())
columns_day_1 = [f"{time}_1_Power(mW)" for time in time_min_groups_sorted]
columns_day_2 = [f"{time}_2_Power(mW)" for time in time_min_groups_sorted]

# 初始化一個新的 DataFrame 來存儲重整後的數據
reshaped_data = []

# 處理每個 LocationCode
for location in expanded_df['LocationCode'].unique():
    location_data = expanded_df[expanded_df['LocationCode'] == location].sort_values(by=['Date', 'TimeMinGroup'])
    unique_dates = sorted(location_data['Date'].unique())
    
    # 迴圈處理每一對連續的兩天
    for i in range(len(unique_dates) - 1):
        day_1 = unique_dates[i]
        day_2 = unique_dates[i + 1]
        
        # 過濾出兩天的數據
        day_1_data = location_data[location_data['Date'] == day_1].set_index('TimeMinGroup')['Power(mW)']
        day_2_data = location_data[location_data['Date'] == day_2].set_index('TimeMinGroup')['Power(mW)']
        
        # 建立一行包含兩天的資料
        combined_row = {
            'LocationCode': location,
            'Date_Day1': day_1,
            'Date_Day2': day_2
        }
        
        # 將每個時間段的資料加入
        combined_row.update({col: day_1_data.get(time, np.nan) for col, time in zip(columns_day_1, time_min_groups_sorted)})
        combined_row.update({col: day_2_data.get(time, np.nan) for col, time in zip(columns_day_2, time_min_groups_sorted)})
        
        # 將每一行的資料加入到列表中
        reshaped_data.append(combined_row)

# 使用 pd.concat 來轉換為 DataFrame
reshaped_data_df = pd.DataFrame(reshaped_data)


In [55]:
#將reshaped_data_df有缺失值的row列出，並存成missing_data
missing_data = reshaped_data_df[reshaped_data_df.isnull().any(axis=1)]
missing_data.shape

#將missing_data存成csv檔
missing_data.to_csv(f'./data/preprocess_data/PowerData{name}_na.csv',index=False)

In [56]:
#將reshaped_data_df中有缺失值的row刪除，並存成csv檔
reshaped_data_df.dropna(inplace=True)
# 檢查欄位名稱
column_names = reshaped_data_df.columns.tolist()
print(column_names)

reshaped_data_df.to_csv(f'./data/preprocess_data/PowerData{name}.csv',index=False)

['LocationCode', 'Date_Day1', 'Date_Day2', '07:00_1_Power(mW)', '07:10_1_Power(mW)', '07:20_1_Power(mW)', '07:30_1_Power(mW)', '07:40_1_Power(mW)', '07:50_1_Power(mW)', '08:00_1_Power(mW)', '08:10_1_Power(mW)', '08:20_1_Power(mW)', '08:30_1_Power(mW)', '08:40_1_Power(mW)', '08:50_1_Power(mW)', '09:00_1_Power(mW)', '09:10_1_Power(mW)', '09:20_1_Power(mW)', '09:30_1_Power(mW)', '09:40_1_Power(mW)', '09:50_1_Power(mW)', '10:00_1_Power(mW)', '10:10_1_Power(mW)', '10:20_1_Power(mW)', '10:30_1_Power(mW)', '10:40_1_Power(mW)', '10:50_1_Power(mW)', '11:00_1_Power(mW)', '11:10_1_Power(mW)', '11:20_1_Power(mW)', '11:30_1_Power(mW)', '11:40_1_Power(mW)', '11:50_1_Power(mW)', '12:00_1_Power(mW)', '12:10_1_Power(mW)', '12:20_1_Power(mW)', '12:30_1_Power(mW)', '12:40_1_Power(mW)', '12:50_1_Power(mW)', '13:00_1_Power(mW)', '13:10_1_Power(mW)', '13:20_1_Power(mW)', '13:30_1_Power(mW)', '13:40_1_Power(mW)', '13:50_1_Power(mW)', '14:00_1_Power(mW)', '14:10_1_Power(mW)', '14:20_1_Power(mW)', '14:30_1_Pow

將PowerData{name}_na整理成可以當測試集的格式

In [57]:
data_na = pd.read_csv(f'./data/preprocess_data/PowerData{name}_na.csv')

#把data_na中col4-col75欄位為nan的資料刪除
data_na = data_na.dropna(subset=data_na.columns[3:75])

#把data_na中16:10_2_Power(mW)欄位是nan的資料列出，並叫data_na_1
data_na_1 = data_na[data_na['16:10_2_Power(mW)'].isnull()]

#取data_na_1_onehot前75欄，後48欄刪除
data_na_1_onehot = data_na_1.iloc[:, 0:75]
print("Feature count of new data:", data_na_1_onehot.shape)

Feature count of new data: (257, 75)


In [58]:
# 檢查欄位名稱
column_names = data_na_1_onehot.columns.tolist()
print(column_names)

data_na_1_onehot.to_csv(f'./data/preprocess_data/PowerData{name}_na_testset_1.csv', index=False)

['LocationCode', 'Date_Day1', 'Date_Day2', '07:00_1_Power(mW)', '07:10_1_Power(mW)', '07:20_1_Power(mW)', '07:30_1_Power(mW)', '07:40_1_Power(mW)', '07:50_1_Power(mW)', '08:00_1_Power(mW)', '08:10_1_Power(mW)', '08:20_1_Power(mW)', '08:30_1_Power(mW)', '08:40_1_Power(mW)', '08:50_1_Power(mW)', '09:00_1_Power(mW)', '09:10_1_Power(mW)', '09:20_1_Power(mW)', '09:30_1_Power(mW)', '09:40_1_Power(mW)', '09:50_1_Power(mW)', '10:00_1_Power(mW)', '10:10_1_Power(mW)', '10:20_1_Power(mW)', '10:30_1_Power(mW)', '10:40_1_Power(mW)', '10:50_1_Power(mW)', '11:00_1_Power(mW)', '11:10_1_Power(mW)', '11:20_1_Power(mW)', '11:30_1_Power(mW)', '11:40_1_Power(mW)', '11:50_1_Power(mW)', '12:00_1_Power(mW)', '12:10_1_Power(mW)', '12:20_1_Power(mW)', '12:30_1_Power(mW)', '12:40_1_Power(mW)', '12:50_1_Power(mW)', '13:00_1_Power(mW)', '13:10_1_Power(mW)', '13:20_1_Power(mW)', '13:30_1_Power(mW)', '13:40_1_Power(mW)', '13:50_1_Power(mW)', '14:00_1_Power(mW)', '14:10_1_Power(mW)', '14:20_1_Power(mW)', '14:30_1_Pow

針對每一個'LocationCode'，從'Date'中的最小值開始到最大值，將每一天的每一個'TimeMinGroup'的'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', Sunlight(Lux), 'Power(mW)'重新整理成每個row都是一筆連續兩天每個時段的值

In [59]:
#按時間排列變數欄位（時間優先），即每個時間點包含四個變數，並按照時間點順序排列
# 定義要處理的變數
variables = ['Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)']

# 取得 TimeMinGroup 排序並生成時間優先的欄位名稱
time_min_groups_sorted = sorted(expanded_df['TimeMinGroup'].unique())
columns_day_1 = [f"{time}_1_{var}" for time in time_min_groups_sorted for var in variables]
columns_day_2 = [f"{time}_2_{var}" for time in time_min_groups_sorted for var in variables]
all_columns = columns_day_1 + columns_day_2  # Day 1 欄位接 Day 2 欄位

# 初始化一個新的列表來存儲重整後的數據
reshaped_data_all = []

# 處理每個 LocationCode
for location in expanded_df['LocationCode'].unique():
    location_data = expanded_df[expanded_df['LocationCode'] == location].sort_values(by=['Date', 'TimeMinGroup'])
    unique_dates = sorted(location_data['Date'].unique())
    
    # 迴圈處理每一對連續的兩天
    for i in range(len(unique_dates) - 1):
        day_1 = unique_dates[i]
        day_2 = unique_dates[i + 1]
        
        # 建立一行包含兩天的資料
        combined_row = {
            'LocationCode': location,
            'Date_Day1': day_1,
            'Date_Day2': day_2
        }
        
        # 逐個時間段及變數加入數據
        for time in time_min_groups_sorted:
            day_1_data = location_data[location_data['Date'] == day_1].set_index('TimeMinGroup')
            day_2_data = location_data[location_data['Date'] == day_2].set_index('TimeMinGroup')
            
            combined_row.update({f"{time}_1_{var}": day_1_data[var].get(time, np.nan) for var in variables})
            combined_row.update({f"{time}_2_{var}": day_2_data[var].get(time, np.nan) for var in variables})
        
        # 添加到列表中
        reshaped_data_all.append(combined_row)

# 使用指定的欄位順序轉換為 DataFrame
reshaped_data_df_all = pd.DataFrame(reshaped_data_all, columns=['LocationCode', 'Date_Day1', 'Date_Day2'] + all_columns)

In [60]:
#將reshaped_data_df_all有缺失值的row列出，並存成missing_data
missing_data = reshaped_data_df_all[reshaped_data_df_all.isnull().any(axis=1)]
missing_data.shape

missing_data.to_csv(f'./data/preprocess_data/AllVariablesData{name}_na.csv',index=False)

In [61]:
#將reshaped_data_df_all中有缺失值的row刪除，並存成csv檔
reshaped_data_df_all.dropna(inplace=True)
# 檢查欄位名稱
column_names = reshaped_data_df_all.columns.tolist()
print(column_names)

reshaped_data_df_all.to_csv(f'./data/preprocess_data/AllVariablesData{name}.csv',index=False)

['LocationCode', 'Date_Day1', 'Date_Day2', '07:00_1_Pressure(hpa)', '07:00_1_Temperature(°C)', '07:00_1_Humidity(%)', '07:00_1_Sunlight(Lux)', '07:00_1_Power(mW)', '07:10_1_Pressure(hpa)', '07:10_1_Temperature(°C)', '07:10_1_Humidity(%)', '07:10_1_Sunlight(Lux)', '07:10_1_Power(mW)', '07:20_1_Pressure(hpa)', '07:20_1_Temperature(°C)', '07:20_1_Humidity(%)', '07:20_1_Sunlight(Lux)', '07:20_1_Power(mW)', '07:30_1_Pressure(hpa)', '07:30_1_Temperature(°C)', '07:30_1_Humidity(%)', '07:30_1_Sunlight(Lux)', '07:30_1_Power(mW)', '07:40_1_Pressure(hpa)', '07:40_1_Temperature(°C)', '07:40_1_Humidity(%)', '07:40_1_Sunlight(Lux)', '07:40_1_Power(mW)', '07:50_1_Pressure(hpa)', '07:50_1_Temperature(°C)', '07:50_1_Humidity(%)', '07:50_1_Sunlight(Lux)', '07:50_1_Power(mW)', '08:00_1_Pressure(hpa)', '08:00_1_Temperature(°C)', '08:00_1_Humidity(%)', '08:00_1_Sunlight(Lux)', '08:00_1_Power(mW)', '08:10_1_Pressure(hpa)', '08:10_1_Temperature(°C)', '08:10_1_Humidity(%)', '08:10_1_Sunlight(Lux)', '08:10_1_P

將AllVariablesData{name}_na整理成可以當測試集的格式

In [62]:
data_na = pd.read_csv(f'./data/preprocess_data/AllVariablesData{name}_na.csv')

#把data_na中col4-col363欄位為nan的資料刪除
data_na = data_na.dropna(subset=data_na.columns[3:363])

#把data_na中16:10_2_Power(mW)欄位是nan的資料列出，並叫data_na_1
data_na_1 = data_na[data_na['16:10_2_Power(mW)'].isnull()]

#取data_na_1_onehot前363欄，後240欄刪除
data_na_1_onehot = data_na_1.iloc[:, 0:363]
print("Feature count of new data:", data_na_1_onehot.shape)

Feature count of new data: (257, 363)


In [63]:
# 檢查欄位名稱
column_names = data_na_1_onehot.columns.tolist()
print(column_names)

data_na_1_onehot.to_csv(f'./data/preprocess_data/AllVariablesData{name}_na_testset_1.csv', index=False)

['LocationCode', 'Date_Day1', 'Date_Day2', '07:00_1_Pressure(hpa)', '07:00_1_Temperature(°C)', '07:00_1_Humidity(%)', '07:00_1_Sunlight(Lux)', '07:00_1_Power(mW)', '07:10_1_Pressure(hpa)', '07:10_1_Temperature(°C)', '07:10_1_Humidity(%)', '07:10_1_Sunlight(Lux)', '07:10_1_Power(mW)', '07:20_1_Pressure(hpa)', '07:20_1_Temperature(°C)', '07:20_1_Humidity(%)', '07:20_1_Sunlight(Lux)', '07:20_1_Power(mW)', '07:30_1_Pressure(hpa)', '07:30_1_Temperature(°C)', '07:30_1_Humidity(%)', '07:30_1_Sunlight(Lux)', '07:30_1_Power(mW)', '07:40_1_Pressure(hpa)', '07:40_1_Temperature(°C)', '07:40_1_Humidity(%)', '07:40_1_Sunlight(Lux)', '07:40_1_Power(mW)', '07:50_1_Pressure(hpa)', '07:50_1_Temperature(°C)', '07:50_1_Humidity(%)', '07:50_1_Sunlight(Lux)', '07:50_1_Power(mW)', '08:00_1_Pressure(hpa)', '08:00_1_Temperature(°C)', '08:00_1_Humidity(%)', '08:00_1_Sunlight(Lux)', '08:00_1_Power(mW)', '08:10_1_Pressure(hpa)', '08:10_1_Temperature(°C)', '08:10_1_Humidity(%)', '08:10_1_Sunlight(Lux)', '08:10_1_P

將每一組時間和四個變項(如07:00_1_Pressure(hpa),07:00_1_Temperature(°C),07:00_1_Humidity(%),07:00_1_Sunlight(Lux),07:00_1_Power(mW))的值，放在一個list裡面，只佔一個欄位

In [64]:
# 定義要處理的變數
variables = ['Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)']

# 取得 TimeMinGroup 排序並生成時間優先的欄位名稱
time_min_groups_sorted = sorted(expanded_df['TimeMinGroup'].unique())
columns_day_1 = [f"{time}_1" for time in time_min_groups_sorted]
columns_day_2 = [f"{time}_2" for time in time_min_groups_sorted]
all_columns = columns_day_1 + columns_day_2  # Day 1 columns followed by Day 2 columns

# 初始化一個新的列表來存儲重整後的數據
reshaped_data_all_list = []

# 處理每個 LocationCode
for location in expanded_df['LocationCode'].unique():
    location_data = expanded_df[expanded_df['LocationCode'] == location].sort_values(by=['Date', 'TimeMinGroup'])
    unique_dates = sorted(location_data['Date'].unique())
    
    # 迴圈處理每一對連續的兩天
    for i in range(len(unique_dates) - 1):
        day_1 = unique_dates[i]
        day_2 = unique_dates[i + 1]
        
        # 建立一行包含兩天的資料
        combined_row = {
            'LocationCode': location,
            'Date_Day1': day_1,
            'Date_Day2': day_2
        }
        
        # 逐個時間段及變數加入數據，並將每組時間的四個變數放入一個列表
        for time in time_min_groups_sorted:
            day_1_data = location_data[location_data['Date'] == day_1].set_index('TimeMinGroup')
            day_2_data = location_data[location_data['Date'] == day_2].set_index('TimeMinGroup')
            
            # 組合 Day 1 和 Day 2 各時間段的變數值為列表
            combined_row[f"{time}_1"] = [day_1_data[var].get(time, np.nan) for var in variables]
            combined_row[f"{time}_2"] = [day_2_data[var].get(time, np.nan) for var in variables]
        
        # 將每一行的資料加入到列表中
        reshaped_data_all_list.append(combined_row)

# 使用指定的欄位順序轉換為 DataFrame
reshaped_data_df_all_list = pd.DataFrame(reshaped_data_all_list, columns=['LocationCode', 'Date_Day1', 'Date_Day2'] + all_columns)
reshaped_data_df_all_list.head(3)


,LocationCode,Date_Day1,Date_Day2,07:00_1,07:10_1,07:20_1,07:30_1,07:40_1,07:50_1,08:00_1,...,15:20_2,15:30_2,15:40_2,15:50_2,16:00_2,16:10_2,16:20_2,16:30_2,16:40_2,16:50_2
0,1,2024-01-01,2024-01-02,"[0.6377789874795857, 0.2591973244147157, 0.853...","[0.6382580293957538, 0.2603678929765886, 0.852...","[0.6398693522046811, 0.261371237458194, 0.8439...","[0.6406314643440385, 0.2632107023411371, 0.835...","[0.6417419706042455, 0.2658862876254181, 0.835...","[0.643658138268916, 0.2697324414715719, 0.8327...","[0.6452803483941203, 0.2724080267558529, 0.823...",...,"[0.5645726728361457, 0.3760869565217392, 0.513...","[0.5655307566684808, 0.3732441471571907, 0.526...","[0.5663037561241149, 0.3729096989966555, 0.529...","[0.5666303756124111, 0.3729096989966555, 0.527...","[0.567250952640174, 0.3720735785953177, 0.5286...","[0.5687207403375061, 0.3677257525083612, 0.537...","[0.5686445291235703, 0.3607023411371237, 0.552...","[0.568383233532934, 0.3555183946488294, 0.5626...","[0.5682961350027217, 0.3520066889632107, 0.574...","[0.5690255851932495, 0.3453177257525083, 0.588..."
1,1,2024-01-02,2024-01-03,"[0.6151442569406637, 0.1837792642140468, 1.000...","[0.6168753402286333, 0.2080267558528428, 1.000...","[0.617332607512248, 0.2265886287625417, 1.0000...","[0.6181164942841582, 0.2381270903010033, 1.000...","[0.6187697332607505, 0.2837792642140468, 1.000...","[0.6179096352749044, 0.3379598662207358, 0.895...","[0.6174523679912898, 0.3759197324414716, 0.677...",...,"[0.5800979858464885, 0.2809364548494983, 0.870...","[0.5815568862275444, 0.2792642140468227, 0.864...","[0.5836472509526395, 0.2779264214046822, 0.857...","[0.5853783342406091, 0.2760869565217391, 0.861...","[0.587555797495917, 0.2754180602006689, 0.8667...","[0.5895917256396295, 0.2717391304347826, 0.875...","[0.5909744148067496, 0.270066889632107, 0.8735...","[0.593413173652694, 0.2678929765886287, 0.8839...","[0.5952531301034292, 0.26438127090301, 0.87794...","[0.5974959172563956, 0.2627090301003344, 0.886..."
2,1,2024-01-03,2024-01-04,"[0.5840391943385951, 0.2247491638795986, 1.000...","[0.5845508982035922, 0.2282608695652173, 1.000...","[0.5855198693522045, 0.2317725752508361, 1.000...","[0.5858791507893304, 0.2369565217391304, 1.000...","[0.5868045726728355, 0.2421404682274247, 1.000...","[0.588339684267828, 0.2491638795986622, 1.0000...","[0.5899183451279255, 0.2595317725752508, 1.000...",...,"[0.6045400108873162, 0.2882943143812709, 0.602...","[0.6053130103429503, 0.2876254180602006, 0.614...","[0.606260206859009, 0.2876254180602006, 0.6212...","[0.6074795862819812, 0.2877926421404682, 0.623...","[0.6088078388677187, 0.2892976588628763, 0.627...","[0.609605032359523, 0.2904124860646599, 0.6280...","[0.6107893304300486, 0.2884615384615384, 0.633...","[0.612259118127381, 0.2846153846153846, 0.6446...","[0.6143168209036469, 0.2806020066889632, 0.656...","[0.6163418617310826, 0.2764214046822743, 0.669..."


In [65]:
#將reshaped_data_df_all_list中有缺失值的row刪除，只要list中有缺失就刪除
def remove_rows_with_nan_in_specific_columns(df, columns_to_check):
    # 創建一個布林遮罩，用來標記特定欄位中包含 NaN 的行
    nan_rows_mask = df[columns_to_check].applymap(lambda x: isinstance(x, list) and any(pd.isna(item) for item in x))
    
    # 刪除任何包含 NaN 的行
    df_filtered = df[~nan_rows_mask.any(axis=1)]
    return df_filtered

# 設定需要檢查的欄位
columns_to_check = [
    '07:00_1', '07:10_1', '07:20_1', '07:30_1', '07:40_1', '07:50_1', '08:00_1', '08:10_1', '08:20_1', '08:30_1', '08:40_1', '08:50_1', '09:00_1', '09:10_1', '09:20_1', '09:30_1', '09:40_1', '09:50_1', '10:00_1', '10:10_1', '10:20_1', '10:30_1', '10:40_1', '10:50_1', '11:00_1', '11:10_1', '11:20_1', '11:30_1', '11:40_1', '11:50_1', '12:00_1', '12:10_1', '12:20_1', '12:30_1', '12:40_1', '12:50_1', '13:00_1', '13:10_1', '13:20_1', '13:30_1', '13:40_1', '13:50_1', '14:00_1', '14:10_1', '14:20_1', '14:30_1', '14:40_1', '14:50_1', '15:00_1', '15:10_1', '15:20_1', '15:30_1', '15:40_1', '15:50_1', '16:00_1', '16:10_1', '16:20_1', '16:30_1', '16:40_1', '16:50_1', '07:00_2', '07:10_2', '07:20_2', '07:30_2', '07:40_2', '07:50_2', '08:00_2', '08:10_2', '08:20_2', '08:30_2', '08:40_2', '08:50_2', '09:00_2', '09:10_2', '09:20_2', '09:30_2', '09:40_2', '09:50_2', '10:00_2', '10:10_2', '10:20_2', '10:30_2', '10:40_2', '10:50_2', '11:00_2', '11:10_2', '11:20_2', '11:30_2', '11:40_2', '11:50_2', '12:00_2', '12:10_2', '12:20_2', '12:30_2', '12:40_2', '12:50_2', '13:00_2', '13:10_2', '13:20_2', '13:30_2', '13:40_2', '13:50_2', '14:00_2', '14:10_2', '14:20_2', '14:30_2', '14:40_2', '14:50_2', '15:00_2', '15:10_2', '15:20_2', '15:30_2', '15:40_2', '15:50_2', '16:00_2', '16:10_2', '16:20_2', '16:30_2', '16:40_2', '16:50_2'
]

# 執行過濾
reshaped_data_df_all_list_filtered = remove_rows_with_nan_in_specific_columns(reshaped_data_df_all_list, columns_to_check)

reshaped_data_df_all_list_filtered.shape

/tmp/ipykernel_3027225/3349010322.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  nan_rows_mask = df[columns_to_check].applymap(lambda x: isinstance(x, list) and any(pd.isna(item) for item in x))


(1098, 123)

In [66]:
#把reshaped_data_df_all_list被刪除的row(不在reshaped_data_df_all_list_filtered中)存成missing_data
missing_data = reshaped_data_df_all_list[~reshaped_data_df_all_list.index.isin(reshaped_data_df_all_list_filtered.index)]
# 檢查欄位名稱
column_names = missing_data.columns.tolist()
print(column_names)

missing_data.to_csv(f'./data/preprocess_data/AllVariablesDataList{name}_na.csv',index=False)

['LocationCode', 'Date_Day1', 'Date_Day2', '07:00_1', '07:10_1', '07:20_1', '07:30_1', '07:40_1', '07:50_1', '08:00_1', '08:10_1', '08:20_1', '08:30_1', '08:40_1', '08:50_1', '09:00_1', '09:10_1', '09:20_1', '09:30_1', '09:40_1', '09:50_1', '10:00_1', '10:10_1', '10:20_1', '10:30_1', '10:40_1', '10:50_1', '11:00_1', '11:10_1', '11:20_1', '11:30_1', '11:40_1', '11:50_1', '12:00_1', '12:10_1', '12:20_1', '12:30_1', '12:40_1', '12:50_1', '13:00_1', '13:10_1', '13:20_1', '13:30_1', '13:40_1', '13:50_1', '14:00_1', '14:10_1', '14:20_1', '14:30_1', '14:40_1', '14:50_1', '15:00_1', '15:10_1', '15:20_1', '15:30_1', '15:40_1', '15:50_1', '16:00_1', '16:10_1', '16:20_1', '16:30_1', '16:40_1', '16:50_1', '07:00_2', '07:10_2', '07:20_2', '07:30_2', '07:40_2', '07:50_2', '08:00_2', '08:10_2', '08:20_2', '08:30_2', '08:40_2', '08:50_2', '09:00_2', '09:10_2', '09:20_2', '09:30_2', '09:40_2', '09:50_2', '10:00_2', '10:10_2', '10:20_2', '10:30_2', '10:40_2', '10:50_2', '11:00_2', '11:10_2', '11:20_2', 

In [67]:
#將reshaped_data_df_all_list中有缺失值的row刪除，只要list中有缺失就刪除，並存成csv檔
# 檢查欄位名稱
column_names = reshaped_data_df_all_list.columns.tolist()
print(column_names)

reshaped_data_df_all_list_filtered.to_csv(f'./data/preprocess_data/AllVariablesDataList{name}.csv',index=False)

['LocationCode', 'Date_Day1', 'Date_Day2', '07:00_1', '07:10_1', '07:20_1', '07:30_1', '07:40_1', '07:50_1', '08:00_1', '08:10_1', '08:20_1', '08:30_1', '08:40_1', '08:50_1', '09:00_1', '09:10_1', '09:20_1', '09:30_1', '09:40_1', '09:50_1', '10:00_1', '10:10_1', '10:20_1', '10:30_1', '10:40_1', '10:50_1', '11:00_1', '11:10_1', '11:20_1', '11:30_1', '11:40_1', '11:50_1', '12:00_1', '12:10_1', '12:20_1', '12:30_1', '12:40_1', '12:50_1', '13:00_1', '13:10_1', '13:20_1', '13:30_1', '13:40_1', '13:50_1', '14:00_1', '14:10_1', '14:20_1', '14:30_1', '14:40_1', '14:50_1', '15:00_1', '15:10_1', '15:20_1', '15:30_1', '15:40_1', '15:50_1', '16:00_1', '16:10_1', '16:20_1', '16:30_1', '16:40_1', '16:50_1', '07:00_2', '07:10_2', '07:20_2', '07:30_2', '07:40_2', '07:50_2', '08:00_2', '08:10_2', '08:20_2', '08:30_2', '08:40_2', '08:50_2', '09:00_2', '09:10_2', '09:20_2', '09:30_2', '09:40_2', '09:50_2', '10:00_2', '10:10_2', '10:20_2', '10:30_2', '10:40_2', '10:50_2', '11:00_2', '11:10_2', '11:20_2', 

將AllVariablesDataList_na整理成可以當測試集的格式

In [68]:
#將reshaped_data_df_all_list中有缺失值的row刪除，只要list中有缺失就刪除
def remove_rows_with_nan_in_specific_columns(df, columns_to_check):
    # 創建一個布林遮罩，用來標記特定欄位中包含 NaN 的行
    nan_rows_mask = df[columns_to_check].applymap(lambda x: isinstance(x, list) and any(pd.isna(item) for item in x))
    
    # 刪除任何包含 NaN 的行
    df_filtered = df[~nan_rows_mask.any(axis=1)]
    return df_filtered

# 設定需要檢查的欄位
columns_to_check = [
    '07:00_1', '07:10_1', '07:20_1', '07:30_1', '07:40_1', '07:50_1', '08:00_1', '08:10_1', '08:20_1', '08:30_1', '08:40_1', '08:50_1', '09:00_1', '09:10_1', '09:20_1', '09:30_1', '09:40_1', '09:50_1', '10:00_1', '10:10_1', '10:20_1', '10:30_1', '10:40_1', '10:50_1', '11:00_1', '11:10_1', '11:20_1', '11:30_1', '11:40_1', '11:50_1', '12:00_1', '12:10_1', '12:20_1', '12:30_1', '12:40_1', '12:50_1', '13:00_1', '13:10_1', '13:20_1', '13:30_1', '13:40_1', '13:50_1', '14:00_1', '14:10_1', '14:20_1', '14:30_1', '14:40_1', '14:50_1', '15:00_1', '15:10_1', '15:20_1', '15:30_1', '15:40_1', '15:50_1', '16:00_1', '16:10_1', '16:20_1', '16:30_1', '16:40_1', '16:50_1', '07:00_2', '07:10_2', '07:20_2', '07:30_2', '07:40_2', '07:50_2', '08:00_2', '08:10_2', '08:20_2', '08:30_2', '08:40_2', '08:50_2'
    #, '09:00_2', '09:10_2', '09:20_2', '09:30_2', '09:40_2', '09:50_2', '10:00_2', '10:10_2', '10:20_2', '10:30_2', '10:40_2', '10:50_2', '11:00_2', '11:10_2', '11:20_2', '11:30_2', '11:40_2', '11:50_2', '12:00_2', '12:10_2', '12:20_2', '12:30_2', '12:40_2', '12:50_2', '13:00_2', '13:10_2', '13:20_2', '13:30_2', '13:40_2', '13:50_2', '14:00_2', '14:10_2', '14:20_2', '14:30_2', '14:40_2', '14:50_2', '15:00_2', '15:10_2', '15:20_2', '15:30_2', '15:40_2', '15:50_2', '16:00_2', '16:10_2', '16:20_2', '16:30_2', '16:40_2', '16:50_2'
]

# 執行過濾
missing_data_filtered = remove_rows_with_nan_in_specific_columns(missing_data, columns_to_check)
missing_data_filtered.shape

/tmp/ipykernel_3027225/2807060523.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  nan_rows_mask = df[columns_to_check].applymap(lambda x: isinstance(x, list) and any(pd.isna(item) for item in x))


(299, 123)

In [69]:
#將reshaped_data_df_all_list中有缺失值的row刪除，只要list中有缺失就刪除
def remove_rows_with_nan_in_specific_columns(df, columns_to_check):
    # 創建一個布林遮罩，用來標記特定欄位中包含 NaN 的行
    nan_rows_mask = df[columns_to_check].applymap(lambda x: isinstance(x, list) and any(pd.isna(item) for item in x))
    
    # 刪除任何包含 NaN 的行
    df_filtered = df[~nan_rows_mask.any(axis=1)]
    return df_filtered

# 設定需要檢查的欄位
columns_to_check = [
'16:10_2'
]

# 執行過濾
missing_data_filtered_1 = remove_rows_with_nan_in_specific_columns(missing_data_filtered, columns_to_check)
missing_data_filtered_1.shape

/tmp/ipykernel_3027225/4215313499.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  nan_rows_mask = df[columns_to_check].applymap(lambda x: isinstance(x, list) and any(pd.isna(item) for item in x))


(42, 123)

In [70]:
#把missing_data_filtered被刪除的row(不在missing_data_filtered_1中)存成AllVariablesDataList_maxmin_na_testset
AllVariablesDataList_na_testset = missing_data_filtered[~missing_data_filtered.index.isin(missing_data_filtered_1.index)]
AllVariablesDataList_na_testset.head(3)

,LocationCode,Date_Day1,Date_Day2,07:00_1,07:10_1,07:20_1,07:30_1,07:40_1,07:50_1,08:00_1,...,15:20_2,15:30_2,15:40_2,15:50_2,16:00_2,16:10_2,16:20_2,16:30_2,16:40_2,16:50_2
4,1,2024-01-05,2024-01-06,"[0.6224169842133914, 0.1792642140468227, 1.000...","[0.623124659771366, 0.2070234113712374, 1.0000...","[0.6242025040827434, 0.2411371237458194, 1.000...","[0.624975503538377, 0.2749163879598662, 0.9696...","[0.6262057702776259, 0.2998327759197324, 0.790...","[0.6270440936309194, 0.3254180602006689, 0.677...","[0.6270549809471959, 0.3506688963210702, 0.587...",...,"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
9,1,2024-01-10,2024-01-11,"[0.56984213391399, 0.2693979933110368, 1.00000...","[0.5711703864997272, 0.2710702341137124, 1.000...","[0.572411540555253, 0.2730769230769231, 1.0000...","[0.5728470332063141, 0.2749163879598662, 1.000...","[0.5742297223734344, 0.2760869565217391, 1.000...","[0.5760914534567227, 0.2779264214046822, 1.000...","[0.5778007621121392, 0.281438127090301, 1.0000...",...,"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
12,1,2024-01-13,2024-01-14,"[0.6216984213391392, 0.2008361204013377, 1.000...","[0.623189983669025, 0.2051839464882943, 1.0000...","[0.6245726728361454, 0.2140468227424749, 1.000...","[0.625781164942841, 0.2237458193979933, 1.0000...","[0.6267392487751767, 0.2327759197324414, 1.000...","[0.6279586281981487, 0.2448160535117056, 1.000...","[0.6293195427327157, 0.255685618729097, 1.0000...",...,"[0.6059771366358186, 0.3710702341137123, 0.522...","[0.6059880239520957, 0.3650501672240803, 0.538...","[0.606169479223371, 0.3619472315124489, 0.5466...","[0.6060152422427869, 0.3591137123745819, 0.562...","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"


In [71]:
#取data_na_1_onehot前75欄，後48欄刪除
AllVariablesDataList_na_testset = AllVariablesDataList_na_testset.iloc[:, 0:75]
print("Feature count of new data:", AllVariablesDataList_na_testset.shape)


AllVariablesDataList_na_testset.to_csv(f'./data/preprocess_data/AllVariablesDataList{name}_na_testset_1.csv', index=False)

Feature count of new data: (257, 75)
